In [1]:
%env DDE_BACKEND=tensorflow
import tensorflow as tf
import deepxde as dde

env: DDE_BACKEND=tensorflow


Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
from models.domains import NewtonianParallelPlane
from models.rheology import NewtonianFluid
from models.PINN import PINNParameters
from models.neural_network import NeuralNetwork, ADAMOptimizer, LBFGSOptimizer
from models.data import TrainingData, NeuralNetworkModel

rheology = NewtonianFluid(mu=1, density=1)
domain = NewtonianParallelPlane(distance=1, length=1, inlet_velocity=1, rheology=rheology)
boundary_conditions = domain.build_boundary_conditions()
neural_network = NeuralNetwork(
    adam_optimizer=ADAMOptimizer(iterations=100, report_frequency=100),
    LBFGS_optimizer=LBFGSOptimizer(max_iterations=100, report_frequency=100)
)
pinn_parameters = PINNParameters(
    domain_points=100,
    boundary_points=100,
    number_of_test=100, 
)
net = neural_network.build_net()
training_data = TrainingData(domain=domain, boundary_condition_list=[v for v in boundary_conditions.values()], pinn_parameters=pinn_parameters)
model = NeuralNetworkModel(data=training_data.data, net=net)

In [6]:
import pathlib

model.compile_ADAM_model(learning_rate=neural_network.adam_optimizer.learning_rate)
adam_output_path = pathlib.Path("./out/adam/model_state-100.ckpt.index")
model.model.restore(adam_output_path)

Compiling model...
'compile' took 0.000468 s

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.
